In [24]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader


np.random.seed(0)
torch.manual_seed(0)

t = np.linspace(0, 100, 1000)
data = np.sin(t)

def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(data, seq_length)

trainX = torch.tensor(X[:, :, None], dtype=torch.float32)
trainY = torch.tensor(y[:, None], dtype=torch.float32)
print("trainX shape: ", trainX.shape)
print("trainY shape: ", trainY.shape)

trainX shape:  torch.Size([990, 10, 1])
trainY shape:  torch.Size([990, 1])


In [25]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [26]:

train_dataset = TensorDataset(trainX, trainY)

batch_size = 64  # You can adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=False)

# Example of accessing a batch of data
for inputs, targets in train_loader:
    print(f'Inputs: {inputs.shape}, Targets: {targets.shape}')
    
    break  # Only print the first batch for verification


Inputs: torch.Size([100, 10, 1]), Targets: torch.Size([100, 1])


In [27]:
class LSTM_pt(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM_pt, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        # LSTM cell
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers = self.num_layers, batch_first = True)
        
        # Linear layer for final prediction
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, inputs, cell_state=None, hidden_state=None):
        # Forward pass through the LSTM cell
        if hidden_state is None or cell_state is None:
            hidden_state = torch.zeros(self.num_layers, inputs.size(0), self.hidden_size).to(device)
            cell_state = torch.zeros(self.num_layers, inputs.size(0), self.hidden_size).to(device)
        hidden = (cell_state, hidden_state)
        output, new_memory = self.lstm(inputs, hidden)
        cell_state, hidden_state = new_memory
        output = self.linear(output[:, -1, :])  # Take only the last time step
        return output, cell_state, hidden_state, # Return correct order

In [28]:
input_size = 1
hidden_size = 20
num_layers = 1 # Can be changed to stack multiple LSTM layers!
output_size = 1
dataloader = train_loader


model = LSTM_pt(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [29]:
num_epochs = 100
h0, c0 = None, None

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs, h0, c0 = model(trainX, h0, c0)

    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()

    h0 = h0.detach()
    c0 = c0.detach()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("Predicted output: ", trainY)

Epoch [10/100], Loss: 0.1303
Epoch [20/100], Loss: 0.0578
Epoch [30/100], Loss: 0.0241
Epoch [40/100], Loss: 0.0057
Epoch [50/100], Loss: 0.0015
Epoch [60/100], Loss: 0.0014
Epoch [70/100], Loss: 0.0007
Epoch [80/100], Loss: 0.0004
Epoch [90/100], Loss: 0.0004
Epoch [100/100], Loss: 0.0003
Predicted output:  tensor([[ 8.4201e-01],
        [ 8.9171e-01],
        [ 9.3247e-01],
        [ 9.6391e-01],
        [ 9.8569e-01],
        [ 9.9760e-01],
        [ 9.9953e-01],
        [ 9.9144e-01],
        [ 9.7344e-01],
        [ 9.4568e-01],
        [ 9.0846e-01],
        [ 8.6215e-01],
        [ 8.0720e-01],
        [ 7.4417e-01],
        [ 6.7369e-01],
        [ 5.9647e-01],
        [ 5.1327e-01],
        [ 4.2493e-01],
        [ 3.3235e-01],
        [ 2.3643e-01],
        [ 1.3815e-01],
        [ 3.8480e-02],
        [-6.1572e-02],
        [-1.6101e-01],
        [-2.5883e-01],
        [-3.5406e-01],
        [-4.4575e-01],
        [-5.3297e-01],
        [-6.1486e-01],
        [-6.9059e-01],


In [33]:
print("pred output: ", outputs.shape)
print("true output: ", trainY.shape)


pred output:  torch.Size([990, 1])
true output:  torch.Size([990, 1])
